# Implmentation of a CNN for Fashion MNIST

## Import Libraries

In [2]:
import numpy
from keras.datasets import fashion_mnist
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.utils import np_utils
from dotdict import dotdict


## Setup W&B

In [4]:
from wandb.keras import WandbCallback
import wandb

# Set the notebook name for W&B
%env "WANDB_NOTEBOOK_NAME" "3-Fashion-MNIST-CNN"
wandb.login()


env: "WANDB_NOTEBOOK_NAME"="3-Fashion-MNIST-CNN"


True

## Config Model

In [6]:
config = dotdict.Dict()
# config.img_width = 28
# config.img_height = 28
# config.first_layer_conv_width = 3
# config.first_layer_conv_height = 3
# config.dense_layer_size = 100
# config.epochs = 10

## Load and Normalise Inputs

## Reshape if GreyScale

## One Hot Encode Labels

## Check Shapes of Data & Labels

## Define Model and Init W&B

## Check Model Summary

## Fit Model